In [ ]:
from pathlib import Path
from pathlib import Path
import numpy as np
import pandas as pd
import bs4 as BeautifulSoup
from io import BytesIO
from PIL import Image as pi
from ipywidgets import HBox, VBox, Layout, HTML
from ipywidgets import Image as Image_widget

import drawSvg as draw
import cairosvg

try:
    from fastai.vision import *
    from fastai.metrics import error_rate
    fastai_imported = True
except Exception as ex:
    print('Switch to fastapi-cpu kernel to train or use model.')
    fastai_imported = False

In [ ]:
doc_image = pi.open('/Users/ray/data/PRImA_Layout_Analysis_Dataset/Images/00000086.tif')

In [ ]:
doc_image.size

In [ ]:
!cat /Users/ray/data/PRImA_Layout_Analysis_Dataset/XML/00000086.xml

In [ ]:
with open('/Users/ray/data/PRImA_Layout_Analysis_Dataset/XML/00000086.xml') as fp:
    soup = BeautifulSoup.BeautifulSoup(fp)

In [ ]:
tags = soup.find_all('textregion')

In [ ]:
for tag in tags:
    print(len(tag.find_all('point')))
    print(tag['type'])

In [ ]:
tag.attrs

In [ ]:
width, height = doc_image.size
d = draw.Drawing(width, height, origin = (0, -height))
for tag in tags:
    count = 0
    p = draw.Path(stroke_width=2, stroke='black',
              fill='#eeee00', fill_opacity=0.5)
    for t in tag.find_all('point'):
        x = int(t['x'])
        y = -int(t['y'])
        if count == 0:
            p.M(x, y)
        else:
            p.L(x,y)
        count += 1
    p.Z()
    d.append(p)

In [ ]:
print(d.asSvg())

In [ ]:
path = Path('/Users/ray/data/PRImA_Layout_Analysis_Dataset/Images')
b = BytesIO()
im = pi.open(path / '00000086.tif')
im.save(b, format='png')

In [ ]:
png = cairosvg.svg2png(d.asSvg())
mask = Image_widget(value=png, layout=Layout(width='40%', height='600px', border='1px solid black'))
document_image = Image_widget(value=b.getvalue(), layout=Layout(width='40%', height='600px', border='1px solid black'))
HBox([mask, document_image])

In [ ]:
out = BytesIO()
cairosvg.svg2png(d.asSvg(), write_to=out)
png = pi.open(out)

In [ ]:
background = im.convert("RGBA")
foreground = png.convert("RGBA")

merged = pi.alpha_composite(background, foreground)

In [ ]:
merged